# Capítulo 7 - Condicionando os dados

## Descobrindo o que existe nos dados

In [1]:
from lxml import objectify #analisa arquivos lxml
import pandas as pd

xml = objectify.parse(open('XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))

for i in  range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'],
                  [obj[0].text, obj[1].text,
                  obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    
search = pd.DataFrame.duplicated(df)
print(df)
print()
print(search[search == True])

  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True
3      3   Third    True

3    True
dtype: bool


C:\Users\55279\AppData\Local\Temp\ipykernel_13512\3731539087.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)
C:\Users\55279\AppData\Local\Temp\ipykernel_13512\3731539087.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)
C:\Users\55279\AppData\Local\Temp\ipykernel_13512\3731539087.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)
C:\Users\55279\AppData\Local\Temp\ipykernel_13512\3731539087.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row_s)


Resumindo, o código lê um arquivo XML, extrai os dados contidos nele e cria um DataFrame do pandas a partir desses dados. Ele também verifica se há linhas duplicadas no DataFrame resultante.

## Removendo duplicatas

In [2]:
print(df.drop_duplicates())

  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True


## Manipulando variáveis categóricas

In [3]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'],
                      dtype='category')

car_data = pd.Series(
    pd.Categorical(
        ['Yellow', 'Green', 'Red', 'Blue', 'Purple'],
                    categories=car_colors, ordered=False))

find_entries = pd.isnull(car_data)

print(car_colors)
print()
print(car_data)
print()
print(find_entries[find_entries == True])

0     Blue
1      Red
2    Green
dtype: category
Categories (3, object): ['Blue', 'Green', 'Red']

0      NaN
1    Green
2      Red
3     Blue
4      NaN
dtype: category
Categories (3, object): ['Blue', 'Green', 'Red']

0    True
4    True
dtype: bool


Em resumo, este código mostra como trabalhar com dados de séries categóricas no pandas e como verificar se há valores nulos em uma série categórica usando a função pd.isnull()

### Combinando níveis

In [4]:
car_colors = pd.Series(['Blue', 'Red', 'Green'],
                      dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Green', 'Red', 'Green'],
                    categories=car_colors, ordered=False))

car_data = car_data.cat.set_categories(
    ["Blue", "Red", "Green", "Blue_Red"])

print(car_data.loc[car_data.isin(['Red'])])
car_data.loc[car_data.isin(['Red'])] = 'Blue_Red'
car_data.loc[car_data.isin(['Blue'])] = 'Blue_Red'

car_data = car_data.cat.set_categories(
    ["Green", "Blue_Red"])

print()
print(car_data)


2    Red
4    Red
dtype: category
Categories (4, object): ['Blue', 'Red', 'Green', 'Blue_Red']

0    Blue_Red
1       Green
2    Blue_Red
3       Green
4    Blue_Red
5       Green
dtype: category
Categories (2, object): ['Green', 'Blue_Red']


Como neste exemplo há apenas 1 item Blue, 2 itens Red, mas 3 itens Green, faz-se a junção das categorias Red e Blue em Blue_Red.O código exemplifica como trabalhar com séries categóricas em pandas e como manipulá-las. Ele cria duas séries categóricas: car_colors e car_data. Em seguida, adiciona a categoria "Blue_Red" em car_data, que é uma combinação de "Blue" e "Red". Depois, substitui todos os elementos da série car_data que pertencem às categorias "Red" e "Blue" pela nova categoria "Blue_Red". Por fim, remove as categorias "Blue" e "Red" da série car_data e deixa apenas as categorias "Green" e "Blue_Red". O código serve como exemplo de como manipular séries categóricas em pandas e pode ser adaptado para outras situações.

## Lidando com dados ausentes

### Encontrando os dados ausentes

In [5]:
import numpy as np

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print(s.isnull())

print()
print(s[s.isnull()])

0    False
1    False
2    False
3     True
4    False
5    False
6     True
dtype: bool

3   NaN
6   NaN
dtype: float64


Resumindo, o código verifica quais valores da série 's' são nulos ou ausentes e, em seguida, retorna apenas esses valores em uma nova série.

### Inserindo dados ausentes

In [14]:
from sklearn.impute import SimpleImputer

s = [[1, 2, 3, np.NaN, 5, 6, None]]

imp = SimpleImputer(missing_values=np.nan,   # cria um objeto imp da classe SimpleImputer
                    strategy='mean')         # especifica que os valores faltantes serão preenchidos com a média
imp.fit([[1, 2, 3, 4, 5, 6, 7]])             

x = pd.Series(imp.transform(s).tolist()[0])

print(x)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
6    7.0
dtype: float64


O código trata os valores NaN de um conjunto de dados numéricos, preenchendo-os com a média dos valores existentes.

## Concatenando e Transformando

(Unir dados de várias fontes)

### Adicionando novos casos e variáveis

In [17]:
df = pd.DataFrame({'A': [2,3,1],
                  'B': [1,2,3],
                  'C': [5,3,4]})

df1 = pd.DataFrame({'A': [4],
                  'B': [4],
                  'C': [4]})

df = df.append(df1)                          # adiciona df1 em df
df = df.reset_index(drop = True)             # redefine o índice
print(df)

df.loc[df.last_valid_index() +1] = [5,5,5]   # adiciona mais uma linha ao df
print()
print(df)

df2 = pd.DataFrame({'D': [1,2,3,4,5]})

df = pd.DataFrame.join(df, df2)             # juntas df2 ao df anterior utilizando o método join()
print()
print(df)

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4
4  5  5  5

   A  B  C  D
0  2  1  5  1
1  3  2  3  2
2  1  3  4  3
3  4  4  4  4
4  5  5  5  5


C:\Users\55279\AppData\Local\Temp\ipykernel_13512\2737174428.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


### Removendo dados

In [19]:
df = pd.DataFrame({'A': [2,3,1],
                  'B': [1,2,3],
                  'C': [5,3,4]})

df = df.drop(df.index[[1]])
print(df)

df = df.drop('B', 1)
print()
print(df)

   A  B  C
0  2  1  5
2  1  3  4

   A  C
0  2  5
2  1  4


C:\Users\55279\AppData\Local\Temp\ipykernel_13512\4219053407.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('B', 1)


O código cria um DataFrame df com três colunas (A, B e C) e três linhas. Em seguida, ele usa o método drop para remover a segunda linha (índice 1) do DataFrame. Isso é feito passando uma lista contendo o índice da linha que deseja ser excluída para o método index.

Depois, é usado novamente o método drop, dessa vez para excluir a coluna "B". A exclusão é feita passando o nome da coluna e o parâmetro 1, que indica que a coluna deve ser excluída. O resultado é um novo DataFrame com apenas as colunas "A" e "C".

### Classificando e misturando

(Ordem e desordem dos dados)

In [23]:
df = pd.DataFrame({'A': [2,1,2,3,3,5,4],
                  'B': [1,2,3,5,4,2,5],
                  'C': [5,3,4,1,1,2,3]})

#ordena o DataFrame pelos valores da coluna A em ordem crescente e, em caso de valores iguais, pela coluna B em ordem crescente
df = df.sort_values(by=['A', 'B'], ascending=[True, True])    

df = df.reset_index(drop=True)       #reseta o índice
print(df)

index = df.index.tolist()
np.random.shuffle(index)
df.loc[df.index[index]]
df = df.reset_index(drop=True)
print()
print(df)


   A  B  C
0  1  2  3
1  2  1  5
2  2  3  4
3  3  4  1
4  3  5  1
5  4  5  3
6  5  2  2

   A  B  C
0  1  2  3
1  2  1  5
2  2  3  4
3  3  4  1
4  3  5  1
5  4  5  3
6  5  2  2
